In [9]:
%conda install pandas sklearn numpy

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - sklearn

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [10]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [11]:
dataset = "titanic"
dataset_directory = f"../../datasets/{dataset}"
training_dataset_directory = f"{dataset_directory}/train.csv"
test_dataset_directory = f"{dataset_directory}/test.csv"

In [12]:
titanic_train_dataframes = pd.read_csv(training_dataset_directory)
print(titanic_train_dataframes)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name     Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    male  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                               Heikkinen, Miss. Laina  female  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                             Allen, Mr. William Henry    male  35.0      0   
..                                                 ...     ...   ... 

# Transformations

In [13]:
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
titanic_train_sex_dataframes = one_hot_encoder.fit_transform(titanic_train_dataframes["Sex"].values.reshape(-1, 1))

titanic_train_sex_dataframes.columns = one_hot_encoder.get_feature_names(['Sex'])

print(titanic_train_sex_dataframes.columns)
print(one_hot_encoder.get_feature_names(['Sex']))
titanic_train_sex_dataframes = pd.DataFrame(titanic_train_sex_dataframes.toarray(), columns=titanic_train_sex_dataframes.columns)
print(titanic_train_sex_dataframes)

titanic_train_dataframes.drop(['Sex'] ,axis=1, inplace=True)

titanic_train_dataframes= pd.concat([titanic_train_dataframes, titanic_train_sex_dataframes ], axis=1)
print(titanic_train_dataframes)

['Sex_female' 'Sex_male']
['Sex_female' 'Sex_male']
     Sex_female  Sex_male
0           0.0       1.0
1           1.0       0.0
2           1.0       0.0
3           1.0       0.0
4           0.0       1.0
..          ...       ...
886         0.0       1.0
887         1.0       0.0
888         1.0       0.0
889         0.0       1.0
890         0.0       1.0

[891 rows x 2 columns]
     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name   Age  SibSp  Parch  \
0                              Braund, Mr. Owen Harris  22.0      1      0   